# SRL Task
의미역 인식(Semantic Role Labeling, SRL)은 자연어 텍스트에서 하나의 동사(predicate)가 어떤 argument를 가지고 있는지 찾아내는 작업입니다.

개체명 인식과 비슷하게 각각의 argument에 대한 span을 BIO로 태깅하고, argument type을 맞춰야 합니다. 또한, 한 문장 안에 있는 predicate가 여러 개일 경우, 각각의 predicate에 대해 서로 다른 argument들을 찾아내어야 합니다.

<img src="files/srl1.PNG">

이 task 역시 개체명 인식에서 했던 것처럼 Word embedding 기반의 Bi-directional LSTM을 사용하여 구현해 보겠습니다.

#### import modules

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from seqeval.metrics import f1_score
from tqdm import tqdm
import torch.nn.utils.rnn as rnn
from data import parse_srl, one_hot

#### device information

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### SRL model
개체명 인식 모듈과 동일하게, Word embedding을 사용하고 LSTM을 통과시키는 모델을 만들어 보겠습니다. 하지만 이번에는 각 문장에서 predicate의 위치를 1로 표시하는 one-hot vector를 추가하여, predicate에 따라 argument를 다르게 예측할 수 있게 만들었습니다.(`forward`함수의 `predicate_input`인자)
<img src="files/srl2.PNG">

In [3]:
class SRLModel(nn.Module):
    def __init__(self, we, hidden_size, tag_size):
        super(SRLModel, self).__init__()
        self.we = nn.Embedding.from_pretrained(torch.FloatTensor(we))
        self.lstm = nn.LSTM(input_size=self.we.embedding_dim+1, hidden_size=hidden_size, num_layers=1, batch_first=True, bidirectional=True)
        self.ffnn = nn.Linear(hidden_size * 2, tag_size)
    
    def forward(self, word_input, predicate_input, word_lens, labels=None):
        we = self.we(word_input)
        predicate_input = torch.unsqueeze(predicate_input, 2)
        lstm_input = torch.cat((we, predicate_input), dim=-1)
        lstm_input = rnn.pack_padded_sequence(lstm_input, word_lens, batch_first=True, enforce_sorted=False)
        out, _ = self.lstm(lstm_input)
        out, output_lens = rnn.pad_packed_sequence(out, batch_first=True)
        out = F.dropout(out)
        pred = self.ffnn(out)
        
        if labels is not None:
            pb = torch.zeros(torch.sum(output_lens), pred.size()[-1])
            lb = torch.zeros(torch.sum(output_lens), dtype=torch.long)
            lsum = 0
            for p, la, le in zip(pred, labels, output_lens):
                pb[lsum:lsum+le] = p[:le, :]
                lb[lsum:lsum+le] = la[:le]
                lsum += le
            pred = pb
            labels = lb
            
            loss = F.cross_entropy(pred, labels)
            return loss
        else:
            pred = F.softmax(pred, dim=-1)
            pred = torch.argmax(pred, dim=-1)
            return pred

### Dataset Generation
NER과 같은 구조체를 사용하여 dataset을 만들어 봅니다. NER과 다른 점은, 하나의 문장에 여러 predicate가 있을 수 있다는 점과, label이 predicate마다 하나씩 있다는 것입니다. 결론적으로, 하나의 문장에 있는 token마다 여러 개의 label을 가질 수 있다는 것입니다. 

또한, predicate의 위치를 나타내는 정보가 필요합니다.

In [4]:
class SRLDataElement:
    def __init__(self, tokens, predicates, args):
        # tokens: list of str
        # predicates: list of int: location of predicates
        # args: list of (list of str): argument tag of tokens, each related to predicate
        assert len(predicates) == len(args)
        self.tokens = tokens
        self.predicates = predicates
        self.args = args
    def __len__(self):
        return len(self.predicates)
    def __getitem__(self, index):
        assert type(index) is int
        return self.tokens, self.args[index], self.predicates[index]
    def token_len(self):
        return len(self.tokens)
    def __iter__(self):
        for i in range(len(self)):
            yield self[i]

In [6]:
class SRLDataset(Dataset):
    def __init__(self, tokens, predicates, labels, token_index_dict, tag_index_dict):
        self.data = []
        error_count = 0
        for t, p, l in zip(tokens, predicates, labels):
            try:
                self.data.append(SRLDataElement(t, p, [*zip(*l)]))
            except:
                error_count += 1
        print("Errors: %d" % error_count)
        self.token2i = token_index_dict
        self.tag2i = tag_index_dict
        self.maxlen = max(map(SRLDataElement.token_len, self.data))
        self.ld = {}
        self.lbuf = 0
        print(self.maxlen)
        for d in self.data:
            self.ld[self.lbuf] = d
            self.lbuf += len(d)
    def __len__(self):
        return self.lbuf

    def __getitem__(self, index):
        target = self.data[0]
        imod = 0
        for i, d in self.ld.items():
            if i > index: break
            target = d
            imod = i
        index -= imod
        token, arg, pred_loc = target[index]
        l = len(token)
        return torch.tensor([self.token2i[x] if x in self.token2i else 0 for x in token] + ([0] * (self.maxlen - l))), \
               torch.tensor([self.tag2i[x] for x in arg] + ([0] * (self.maxlen - l))), \
               torch.tensor(one_hot(pred_loc, self.maxlen)).to(dtype=torch.float32), \
               l
    def __iter__(self):
        for d in self.data:
            for x in d:
                yield x

#### Load corpus
NER과 같은 방식으로 데이터를 불러옵니다.

In [7]:
tok_ids = {"UNK_": 0}
with open("wiki_tok_glove_300.word", encoding="UTF8") as f:
    for line in f.readlines():
        tok_ids[line.strip()] = len(tok_ids)
we = np.load("wiki_tok_glove_300.npy")
we = np.vstack([np.zeros([1, we.shape[1]]), we])

In [8]:
tt, tp, tl = parse_srl("corpus/srl_train.conll")
dt, dp, dl = parse_srl("corpus/srl_test.conll")
lset = set([])
for l in tl+dl:
    for ll in l:
        for label in ll:
            lset.add(label)
ldict = {}
for i, l in enumerate(lset):
    ldict[l] = i
i2tag = {v: k for k, v in ldict.items()}
train_dataset = SRLDataset(tt, tp, tl, tok_ids, ldict)
test_dataset = SRLDataset(dt, dp, dl, tok_ids, ldict)

Errors: 1
55
Errors: 0
47


### Training

위에서 만든 SRL 모듈의 instance를 만들고, 이를 optimizer에 등록한 뒤, NER과 같은 방식으로 train합니다.

In [9]:
srl_module = SRLModel(we, 256, len(ldict)).to(device)
optimizer = torch.optim.Adam(srl_module.parameters(), lr=1e-4)

In [10]:
max_epoch = 1000
eval_per_epoch = 5

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=32)

best_f1 = 0
best_epoch = 0
nbc = 0
# ner_parallel = nn.DataParallel(ner_module)
tq = tqdm(range(1, max_epoch+1))
for epoch in tq:
    srl_module.train()
    total_loss = 0
    for train_elem in train_dataloader:
        optimizer.zero_grad()
        
        token_batch, tag_batch, pred_batch, data_len = [tensor.to(device) for tensor in train_elem]
        max_data_len = torch.max(data_len)
        loss = srl_module(token_batch, pred_batch, data_len, tag_batch)
        total_loss += loss
        loss.backward()
        optimizer.step()
    tq.desc = "Epoch %d Loss %f" % (epoch, total_loss)
    if epoch % eval_per_epoch == 0:
        srl_module.eval()
        preds = []
        golds = []
        lens_with_pad = []
        data_lens = []
        for token_batch, tag_batch, pred_batch, data_len in test_dataloader:
            
            token_batch, pred_batch, data_len = token_batch.to(device), pred_batch.to(device), data_len.to(device)
            pred = srl_module(token_batch, pred_batch, data_len)
            for p, t, d in zip(pred, tag_batch, data_len):
                preds.append([i2tag[x.item()] for x in p[:d]])
                golds.append([i2tag[x.item()] for x in t[:d]])
                data_lens.append(d)
        f1 = f1_score(golds, preds)
        if f1 > best_f1:
            best_f1 = f1
            best_epoch = epoch
            torch.save(srl_module.state_dict(), "srl_model")
        
        idx = 0
        with open("srl_debug/eval_%d.tsv" % epoch, "w", encoding="UTF8") as f:
            for token, tag, predicate_loc in test_dataset:
                predicate_loc = one_hot(predicate_loc, len(token))
                for i, (tok, t, predloc) in enumerate(zip(token, tag, predicate_loc)):
                    assert golds[idx][i] == t
                    f.write("\t".join([tok, str(predloc), t, preds[idx][i]])+"\n")
                f.write("\n")
                idx += 1
        print("Epoch %d F1 score %.2f" % (epoch, f1 * 100))
        print("Best F1 %.2f at Epoch %d" % (best_f1 * 100, best_epoch))
        if epoch - best_epoch >= 30:
            print("No better result since epoch %d - stop training" % best_epoch)
            break

Epoch 5 Loss 233.364365:   0%|          | 5/1000 [01:54<6:35:52, 23.87s/it]

Epoch 5 F1 score 31.06
Best F1 31.06 at Epoch 5


Epoch 10 Loss 202.115875:   1%|          | 10/1000 [03:50<6:45:07, 24.55s/it]

Epoch 10 F1 score 36.77
Best F1 36.77 at Epoch 10


Epoch 15 Loss 193.700775:   2%|▏         | 15/1000 [05:47<6:42:01, 24.49s/it]

...

Epoch 70 F1 score 39.32
Best F1 40.16 at Epoch 50


Epoch 75 Loss 144.760468:   8%|▊         | 75/1000 [28:35<5:58:33, 23.26s/it]

Epoch 75 F1 score 39.03
Best F1 40.16 at Epoch 50


Epoch 79 Loss 141.496765:   8%|▊         | 79/1000 [29:59<5:32:02, 21.63s/it]

Epoch 80 F1 score 38.83
Best F1 40.16 at Epoch 50
No better result since epoch 50 - stop training
